# L3 M2.3: Encryption & Secrets Management

## Learning Arc

**Purpose:** Build a production-ready secrets management system for multi-tenant RAG serving 50+ business units across 3 regions (India, US, EU) under GCC compliance framework (SOX 404, DPDPA, GDPR).

**Concepts Covered:**
1. **Secrets Management:** HashiCorp Vault integration with Kubernetes ServiceAccount authentication for dynamic credential retrieval (no hardcoded secrets)
2. **Encryption at Rest:** AES-256 envelope encryption for Pinecone vectors, PostgreSQL metadata, and Redis caches
3. **Encryption in Transit:** TLS 1.3 certificate management with automated renewal via cert-manager + Let's Encrypt
4. **Key Rotation:** Automated quarterly rotation for API keys (OpenAI, Pinecone) and daily rotation for database credentials
5. **GCC Compliance:** SOX 404 (7-year immutable audit trails), DPDPA (India data residency), GDPR (EU data protection)
6. **Failure Prevention:** Detection and mitigation of common security failures (hardcoded secrets, unencrypted backups, missed rotations)

**After Completing This Notebook:**
- You will understand how to integrate HashiCorp Vault for dynamic secrets retrieval with pod identity verification
- You can implement AES-256 envelope encryption with KEK stored in Vault
- You will configure TLS 1.3 with automated certificate renewal (zero-downtime rotation)
- You can build automated key rotation workflows (quarterly for APIs, daily for DB credentials)
- You will recognize critical security failures and implement SOC 2/ISO 27001 compliance controls
- You can deploy multi-region Vault clusters for data residency compliance (DPDPA, GDPR)

**Context in Track L3.M2:**
This module builds on **GCC Compliance M2.1-M2.2 (AuthN/AuthZ)** and prepares you for **M2.4 (Rate Limiting & DDoS Protection)**. It's the security foundation for all GCC RAG operations, ensuring zero hardcoded secrets and full encryption compliance.

In [ ]:
# Environment Setup
import os
import sys
import logging

# Add src to path for imports
if '../src' not in sys.path:
    sys.path.insert(0, '../src')
if '..' not in sys.path:
    sys.path.insert(0, '..')

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')
logger = logging.getLogger(__name__)

# OFFLINE mode for L3 consistency (no external Vault/services required)
OFFLINE = os.getenv("OFFLINE", "true").lower() == "true"

# Vault configuration
VAULT_ENABLED = os.getenv("VAULT_ENABLED", "false").lower() == "true"

if OFFLINE or not VAULT_ENABLED:
    print("⚠️  Running in OFFLINE/VAULT_DISABLED mode")
    print("   → External Vault API calls will be skipped")
    print("   → Encryption examples will use demo KEK")
    print("   → Set VAULT_ENABLED=true in .env to enable Vault integration")
    print("")
else:
    print("✓ Vault integration enabled")
    print(f"  Vault Address: {os.getenv('VAULT_ADDR', 'http://localhost:8200')}")
    print("")

## Section 1: Introduction to Secrets Management

### The Problem: Hardcoded Secrets

**Real-World Failure (Bangalore GCC):**
- Developer committed `.env` file with OpenAI API key to public GitHub
- Credentials exposed for **18 months**
- Attacker discovered key via automated scanning
- **$120K** in unauthorized cryptocurrency mining costs
- **$50M Series B funding lost** when investors discovered breach
- CTO and CFO terminated

**The Traditional (Broken) Approach:**
```python
# ❌ NEVER DO THIS
OPENAI_API_KEY = "sk-1234567890abcdef"  # Hardcoded in code
PINECONE_API_KEY = "abc123-xyz789"      # Committed to GitHub
DB_PASSWORD = "postgres123"             # Static, never rotated
```

**Problems:**
1. Secrets in version control (GitHub, Git history)
2. No rotation (same key for months/years)
3. No audit trail (who accessed when?)
4. Manual distribution (email, Slack, etc.)
5. No expiration (compromised keys stay valid)

### The Solution: Dynamic Secrets with Vault

**Hotel Key-Card Analogy:**
- **Traditional (Hardcoded):** Master key copied to all rooms → security nightmare
- **Vault (Dynamic):** Time-limited key-card per guest → expires 24 hours
- **Benefits:** Guest checkout = automatic invalidation, lost card = reissue without changing locks

**Vault Workflow:**
1. Pod starts with Kubernetes ServiceAccount token (proves identity)
2. Vault verifies token with K8s API server
3. Vault returns client token (24-hour TTL)
4. Application retrieves secrets via client token
5. Pod terminates → token invalidated automatically

**Key Properties:**
- ✅ Zero hardcoded secrets (all from Vault)
- ✅ Time-limited credentials (24-hour TTL)
- ✅ Automatic rotation (daily for DB, quarterly for APIs)
- ✅ Audit trail (all access logged)
- ✅ Pod identity (K8s ServiceAccount verification)

In [ ]:
# Import VaultClient
from l3_m2_security_access_control import VaultClient

# Expected: VaultClient class imported successfully
print(f"✓ VaultClient imported: {VaultClient}")
print("\nVaultClient capabilities:")
print("  - Kubernetes ServiceAccount authentication")
print("  - KV v2 secrets engine support")
print("  - Database dynamic secrets engine")
print("  - Automatic token renewal")
print("  - Singleton pattern (single connection)")

## Section 2: Vault Integration & Authentication

### Kubernetes ServiceAccount Authentication

**Authentication Flow:**
```
┌─────────────┐
│ RAG API Pod │
│ (K8s SA:    │
│  rag-api)   │
└──────┬──────┘
       │ 1. Read JWT token from
       │    /var/run/secrets/kubernetes.io/serviceaccount/token
       ▼
┌─────────────┐
│   Vault     │ 2. Verify JWT with K8s API server
│   Server    │ 3. Check ServiceAccount "rag-api" allowed
└──────┬──────┘ 4. Return client token (24h TTL)
       │
       ▼
┌─────────────┐
│ VaultClient │ 5. Use client token for secret requests
│  (in pod)   │
└─────────────┘
```

**Security Properties:**
- Pod identity verified by K8s API server (cryptographic proof)
- No static tokens in environment variables
- Token expires in 24 hours (automatic renewal)
- Pod termination = token invalidation (cleanup)
- RBAC policies limit secret access per service

**Fallback: Static Token (Development Only)**
For local development without Kubernetes, VaultClient falls back to static `VAULT_TOKEN` environment variable. **NEVER use in production.**

In [ ]:
# Vault Authentication Demo (Offline Mode)

if VAULT_ENABLED and not OFFLINE:
    # Production: Kubernetes ServiceAccount auth
    vault_addr = os.getenv("VAULT_ADDR", "http://localhost:8200")
    k8s_role = os.getenv("VAULT_K8S_ROLE", "rag-api")
    k8s_sa_token_path = os.getenv(
        "VAULT_K8S_SA_TOKEN_PATH",
        "/var/run/secrets/kubernetes.io/serviceaccount/token"
    )
    
    try:
        vault = VaultClient(
            vault_addr=vault_addr,
            k8s_role=k8s_role,
            k8s_sa_token_path=k8s_sa_token_path
        )
        print(f"✓ Vault authenticated via K8s ServiceAccount: {k8s_role}")
    except Exception as e:
        print(f"⚠️  K8s auth failed, trying static token: {e}")
        vault_token = os.getenv("VAULT_TOKEN")
        if vault_token:
            vault = VaultClient(vault_addr=vault_addr, token=vault_token)
            print("✓ Vault authenticated via static token (DEV MODE)")
        else:
            print("❌ No authentication method available")
else:
    print("⚠️  OFFLINE mode - skipping Vault authentication")
    print("")
    print("In production, Vault authentication flow:")
    print("1. Pod reads K8s ServiceAccount JWT token")
    print("2. VaultClient sends JWT to Vault")
    print("3. Vault verifies with K8s API server")
    print("4. Vault returns client token (24-hour TTL)")
    print("5. Client token used for all secret requests")

# Expected: Authentication successful or skipped in offline mode

## Section 3: Retrieving Secrets from Vault

### KV v2 Secrets Engine

**Secret Paths in GCC System:**
- `gcc-secrets/openai` → OpenAI API key + organization
- `gcc-secrets/pinecone` → Pinecone API key + environment
- `gcc-secrets/postgres` → PostgreSQL credentials (dynamic, 24-hour TTL)
- `gcc-secrets/encryption/kek` → Key Encryption Key for AES-256

**Secret Structure (KV v2):**
```json
{
  "data": {
    "data": {
      "api_key": "sk-...",
      "organization": "org-..."
    },
    "metadata": {
      "created_time": "2025-01-15T10:00:00Z",
      "version": 3
    }
  }
}
```

**Security:**
- VaultClient never logs secret values (only paths)
- RBAC policies restrict access per service
- All access logged to audit trail (S3 Glacier)
- Secrets versioned (can rollback if needed)

In [ ]:
# Secret Retrieval Demo

if VAULT_ENABLED and not OFFLINE:
    try:
        # Retrieve OpenAI credentials
        openai_key = vault.get_openai_key()
        print(f"✓ OpenAI API key retrieved: {openai_key[:10]}...")
        
        # Retrieve Pinecone credentials
        pinecone_key = vault.get_pinecone_key()
        print(f"✓ Pinecone API key retrieved: {pinecone_key[:10]}...")
        
        # Retrieve PostgreSQL dynamic credentials
        postgres_creds = vault.get_postgres_credentials()
        print(f"✓ PostgreSQL credentials retrieved:")
        print(f"  Username: {postgres_creds['username']}")
        print(f"  Host: {postgres_creds['host']}")
        print(f"  TTL: 24 hours (auto-rotated daily)")
        
    except Exception as e:
        print(f"⚠️  Secret retrieval failed: {e}")
        print("   Ensure Vault has test secrets populated at:")
        print("   - gcc-secrets/openai")
        print("   - gcc-secrets/pinecone")
        print("   - gcc-secrets/postgres")
else:
    print("⚠️  OFFLINE mode - skipping secret retrieval")
    print("")
    print("In production, secrets retrieved dynamically:")
    print("  openai_key = vault.get_openai_key()")
    print("  pinecone_key = vault.get_pinecone_key()")
    print("  postgres_creds = vault.get_postgres_credentials()")
    print("")
    print("Example PostgreSQL credentials (dynamic):")
    print("  {'username': 'v-k8s-rag-api-abc123',")
    print("   'password': 'A1b2C3d4E5f6',")
    print("   'host': 'postgres.gcc.internal',")
    print("   'port': '5432',")
    print("   'database': 'gcc_rag'}")

# Expected: Secrets retrieved or offline mode explanation

## Section 4: AES-256 Encryption at Rest

### Envelope Encryption Pattern

**Problem:** Encrypting all data with single key = compromise entire dataset if key stolen

**Solution:** Envelope encryption with two-tier key hierarchy

**Key Hierarchy:**
1. **KEK (Key Encryption Key):** Master key stored in Vault, rotated annually
2. **DEK (Data Encryption Key):** Random per-encryption operation, encrypts actual data

**Encryption Process:**
```
┌──────────────┐
│  Plaintext   │
└──────┬───────┘
       │ 1. Generate random DEK (32 bytes)
       │ 2. Generate random IV (16 bytes)
       ▼
┌──────────────┐
│ AES-256-GCM  │ 3. Encrypt with DEK
│  Encryption  │    → ciphertext + auth tag
└──────┬───────┘
       │
       ▼
┌──────────────┐
│  Retrieve    │ 4. Get KEK from Vault
│  KEK from    │
│    Vault     │
└──────┬───────┘
       │ 5. Encrypt DEK with KEK
       ▼
┌──────────────┐
│ Concatenate: │ encrypted_dek || iv || ciphertext || tag
│  Base64      │
└──────────────┘
```

**Benefits:**
- KEK compromise doesn't expose data (need encrypted DEK too)
- KEK rotation = re-encrypt all DEKs (not all data)
- Different DEK per operation (random IV prevents pattern analysis)
- GCM mode = authenticated encryption (confidentiality + integrity)

In [ ]:
# AES-256 Encryption Demo
from l3_m2_security_access_control import EncryptionManager
import os

# Generate demo KEK (in production, retrieve from Vault)
demo_kek = os.urandom(32)  # 256 bits
print(f"✓ Generated demo KEK (32 bytes): {demo_kek.hex()[:20]}...")
print("  (In production: kek = vault.get_secret('gcc-secrets/encryption/kek')['key'])")
print("")

# Initialize EncryptionManager
manager = EncryptionManager(demo_kek)
print("✓ EncryptionManager initialized")
print("")

# Encrypt sensitive PII data
sensitive_data = "User PII: SSN 123-45-6789, Email user@example.com"
print(f"Plaintext: {sensitive_data}")
print("")

encrypted = manager.encrypt(sensitive_data)
print(f"Encrypted (base64): {encrypted[:50]}...")
print(f"  Length: {len(encrypted)} characters")
print("  Structure: encrypted_dek (32) || iv (16) || ciphertext || tag (16)")
print("")

# Decrypt to verify
decrypted = manager.decrypt(encrypted)
print(f"Decrypted: {decrypted}")
print("")

# Verify roundtrip
assert decrypted == sensitive_data, "Decryption failed!"
print("✓ Encryption/decryption roundtrip successful")

# Expected: Encryption works, decryption recovers original plaintext

In [ ]:
# Demonstrate Random IV (Different Ciphertext for Same Plaintext)

plaintext = "Same input data"

encrypted1 = manager.encrypt(plaintext)
encrypted2 = manager.encrypt(plaintext)

print(f"Plaintext: {plaintext}")
print("")
print(f"Encrypted #1: {encrypted1[:50]}...")
print(f"Encrypted #2: {encrypted2[:50]}...")
print("")

print(f"Ciphertexts identical? {encrypted1 == encrypted2}")
print("  (Should be False due to random IV)")
print("")

# Both decrypt to same plaintext
assert manager.decrypt(encrypted1) == plaintext
assert manager.decrypt(encrypted2) == plaintext
print("✓ Both ciphertexts decrypt to original plaintext")
print("")
print("Security benefit: Prevents pattern analysis attacks")
print("  Attacker can't tell if two ciphertexts contain same data")

# Expected: Different ciphertexts, both decrypt correctly

## Section 5: TLS 1.3 Certificate Management

### Encryption in Transit

**Connections Requiring TLS 1.3:**
1. **User → RAG API:** Client requests to FastAPI endpoints
2. **RAG → OpenAI:** Embedding generation requests
3. **RAG → Pinecone:** Vector search queries
4. **RAG → PostgreSQL:** Metadata storage queries

**cert-manager + Let's Encrypt Automation:**
```
┌────────────────┐
│  cert-manager  │ 1. Monitors Certificate resource
│   (K8s pod)    │ 2. Creates ACME challenge
└────────┬───────┘
         │ 3. HTTP-01 or DNS-01 validation
         ▼
┌────────────────┐
│ Let's Encrypt  │ 4. Issues certificate (90-day validity)
└────────┬───────┘
         │ 5. Store in K8s Secret
         ▼
┌────────────────┐
│  Init Container│ 6. Mount Secret to pod
│  (RAG API pod) │    /etc/tls/certs/cert.pem
└────────┬───────┘    /etc/tls/certs/key.pem
         │
         ▼
┌────────────────┐
│  Application   │ 7. Load cert on startup
│  (FastAPI)     │ 8. Serve HTTPS on port 443
└────────────────┘
```

**Automatic Renewal (Zero Downtime):**
- cert-manager renews 30 days before expiry
- Updated certificate stored in K8s Secret
- Application hot-reloads certificate (no restart)
- Old connections continue, new connections use new cert

**Security Properties:**
- TLS 1.3 only (no downgrade to TLS 1.2)
- Forward secrecy (Diffie-Hellman key exchange)
- Mutual TLS support (client certificates)
- Certificate pinning for API calls

In [ ]:
# TLS Certificate Management Demo
from l3_m2_security_access_control import TLSCertificateManager

cert_path = os.getenv("CERT_PATH", "/etc/tls/certs")
print(f"Certificate path: {cert_path}")
print("")

# Initialize manager
tls_manager = TLSCertificateManager(cert_path, renewal_days=30)
print("✓ TLSCertificateManager initialized")
print(f"  Renewal threshold: {tls_manager.renewal_days} days")
print("")

# Check certificate expiry
try:
    days_left = tls_manager.check_expiry()
    print(f"✓ Certificate expires in {days_left} days")
    print("")
    
    if tls_manager.should_renew():
        print("⚠️  Certificate renewal recommended")
        print("   (Less than 30 days until expiry)")
    else:
        print("✓ Certificate valid, no renewal needed")
        
except FileNotFoundError:
    print("⚠️  Certificate not found (expected in offline mode)")
    print("")
    print("In production with cert-manager:")
    print("  1. cert-manager monitors Certificate resource")
    print("  2. Automatic renewal 30 days before expiry")
    print("  3. Certificate stored in K8s Secret")
    print("  4. Init container mounts to /etc/tls/certs")
    print("  5. Application loads cert.pem and key.pem")
    print("")
    print("Certificate lifecycle:")
    print("  Issued: Let's Encrypt")
    print("  Validity: 90 days")
    print("  Renewal: Automated (30 days before expiry)")
    print("  Downtime: Zero (hot reload)")

# Expected: Certificate check or offline mode explanation

## Section 6: Key Rotation Automation

### Rotation Schedules (GCC Standard)

**API Keys (Quarterly - 90 Days):**
- OpenAI API key
- Pinecone API key
- Any third-party service keys

**Database Credentials (Daily - 24 Hours):**
- PostgreSQL via Vault dynamic secrets engine
- Automatic generation on each request
- Old credentials expire after TTL

**Encryption Keys (Annual - 365 Days):**
- KEK (Key Encryption Key) in Vault
- Rotation triggers re-encryption of all DEKs
- Batch job processes all encrypted data

### Rotation Workflow (Zero Downtime)

```
┌────────────────┐
│ KeyRotation    │ 1. Generate new key/credential
│   Manager      │
└────────┬───────┘
         │ 2. Update Vault with new value
         ▼
┌────────────────┐
│     Vault      │ 3. Versioned storage (old + new)
└────────┬───────┘
         │ 4. Grace period (both keys valid)
         ▼
┌────────────────┐
│  Application   │ 5. Next secret request gets new key
│  (RAG API)     │ 6. Existing connections use old key
└────────┬───────┘
         │ 7. After grace period (e.g., 1 hour)
         ▼
┌────────────────┐
│ Invalidate Old │ 8. Old key disabled in Vault
│      Key       │ 9. Log rotation event to audit trail
└────────────────┘
```

**Audit Trail (SOX 404 Compliance):**
Every rotation event logged with:
- Event type (api_key_rotation, postgres_credential_rotation)
- Timestamp (ISO 8601)
- Service (openai, pinecone, postgres)
- Status (success, failed)
- Operator (automated scheduler, admin user)
- Storage: S3 Glacier with 7-year retention + legal hold

In [ ]:
# Key Rotation Demo (Offline Mode)
from l3_m2_security_access_control import KeyRotationManager, audit_log_entry
from datetime import datetime

if VAULT_ENABLED and not OFFLINE:
    # Production rotation
    rotation_manager = KeyRotationManager(vault)
    
    print("Rotating OpenAI API key...")
    new_key = "sk-new-key-2025-01-15-xyz"  # In production, from OpenAI dashboard
    rotation_manager.rotate_api_key("openai", new_key)
    print("✓ OpenAI API key rotated")
    print("")
    
    print("Rotating PostgreSQL credentials...")
    new_creds = rotation_manager.rotate_postgres_credentials()
    print(f"✓ New PostgreSQL credentials:")
    print(f"  Username: {new_creds['username']}")
    print(f"  TTL: 24 hours")
    
else:
    print("⚠️  OFFLINE mode - simulating key rotation")
    print("")
    
    # Simulate rotation audit log
    print("Simulating OpenAI API key rotation:")
    audit_log_entry(
        event_type="api_key_rotation",
        service="openai",
        timestamp=datetime.utcnow().isoformat(),
        status="success",
        previous_key_age_days=85,
        rotation_schedule="quarterly"
    )
    print("  ✓ Old key (85 days old) invalidated")
    print("  ✓ New key active")
    print("  ✓ Audit event logged to S3 Glacier")
    print("")
    
    print("Simulating PostgreSQL credential rotation:")
    audit_log_entry(
        event_type="postgres_credential_rotation",
        timestamp=datetime.utcnow().isoformat(),
        status="success",
        method="vault_dynamic_secrets",
        ttl_hours=24
    )
    print("  ✓ New credentials: v-k8s-rag-api-xyz789")
    print("  ✓ TTL: 24 hours")
    print("  ✓ Old credentials expire automatically")
    print("")
    
    print("Rotation schedules in production:")
    print("  - OpenAI/Pinecone API keys: Every 90 days (quarterly)")
    print("  - PostgreSQL credentials: Every 24 hours (daily)")
    print("  - KEK master key: Every 365 days (annual)")

# Expected: Rotation simulated with audit logging

## Section 7: GCC Compliance Requirements

### Multi-Framework Compliance

GCC operates under **overlapping regulatory frameworks** due to:
- **Parent Company:** US-based (SOX 404)
- **India Operations:** DPDPA compliance
- **Global Clients:** GDPR (EU), CCPA (California)

### SOX 404 (Sarbanes-Oxley)

**Requirement:** 7-year immutable audit trail for all financial system access

**Implementation:**
- All Vault secret access logged with timestamp, user/pod identity, secret path
- Logs sent to S3 Glacier with **legal hold** (prevents deletion)
- Digital signatures for log integrity verification
- Quarterly audit reviews by compliance team

**Audit Events:**
```json
{
  "event_type": "secret_access",
  "timestamp": "2025-01-15T10:30:00Z",
  "user": "rag-api-pod-xyz",
  "secret_path": "gcc-secrets/openai",
  "status": "success",
  "ip_address": "10.0.1.15",
  "k8s_namespace": "production"
}
```

### DPDPA (India Data Protection)

**Requirement:** India user data must reside in India region

**Implementation:**
- Multi-region Vault clusters (India, US, EU)
- Request routing based on user region metadata
- Secrets for India users stored in `vault-india.gcc.internal`
- Cross-region replication for disaster recovery

**Data Flow:**
```
India User Request
  → RAG API (India pod)
    → Vault India cluster
      → PostgreSQL India region
        → Pinecone India index
```

### GDPR (EU Data Protection)

**Requirement:** EU citizen data protection + right to erasure

**Implementation:**
- AES-256 encryption for all EU user PII
- KEK stored in Vault EU cluster
- Data deletion workflow removes encrypted data + KEK
- Audit trail for all EU user data access

**Right to be Forgotten:**
1. User requests data deletion
2. System identifies all encrypted data for user
3. Delete encrypted data from Pinecone/PostgreSQL
4. Invalidate user-specific KEK in Vault
5. Log deletion event to audit trail

### SOC 2 Type II

**Security Principles:**
- **Confidentiality:** AES-256 encryption + Vault secrets
- **Integrity:** GCM authentication tags + audit logs
- **Availability:** HA Vault cluster (3 nodes) + multi-region

**Controls:**
- Automated key rotation (quarterly/daily)
- RBAC policies limit secret access
- Continuous monitoring (Prometheus metrics)
- Incident response procedures

### ISO 27001

**Information Security:**
- Risk assessment: Quarterly Vault security reviews
- Access control: K8s ServiceAccount authentication
- Cryptography: AES-256 at rest, TLS 1.3 in transit
- Supplier relationships: OpenAI/Pinecone SLAs

In [ ]:
# GCC Compliance Audit Trail Demo
import json

print("=" * 60)
print("GCC Compliance Audit Events (7-Year Retention)")
print("=" * 60)
print("")

# Simulate audit events for different compliance frameworks
audit_events = [
    {
        "event_type": "secret_access",
        "timestamp": "2025-01-15T10:30:00Z",
        "user": "rag-api-pod-xyz",
        "secret_path": "gcc-secrets/openai",
        "status": "success",
        "compliance_framework": "SOX_404",
        "region": "us"
    },
    {
        "event_type": "encryption_request",
        "timestamp": "2025-01-15T10:31:15Z",
        "user": "rag-api-pod-xyz",
        "data_type": "user_pii",
        "status": "success",
        "compliance_framework": "GDPR",
        "region": "eu"
    },
    {
        "event_type": "api_key_rotation",
        "timestamp": "2025-01-15T11:00:00Z",
        "service": "openai",
        "status": "success",
        "rotation_schedule": "quarterly",
        "compliance_framework": "SOC_2",
        "region": "us"
    },
    {
        "event_type": "data_residency_check",
        "timestamp": "2025-01-15T11:05:00Z",
        "user_region": "india",
        "vault_cluster": "vault-india.gcc.internal",
        "status": "compliant",
        "compliance_framework": "DPDPA",
        "region": "india"
    }
]

for event in audit_events:
    print(json.dumps(event, indent=2))
    print("")

print("Storage: S3 Glacier (7-year retention, legal hold)")
print("Integrity: Digital signatures (SHA-256)")
print("Access: Compliance team only (read-only)")
print("")

print("Compliance Framework Coverage:")
frameworks = set(e["compliance_framework"] for e in audit_events)
for fw in sorted(frameworks):
    print(f"  ✓ {fw}")

# Expected: Audit events logged for multiple compliance frameworks

## Section 8: Failure Scenarios & Prevention

### Critical Failure Case Studies

#### Failure 1: Hardcoded Secrets (Bangalore GCC)

**What Happened:**
- Developer committed `.env` file with OpenAI API key to **public GitHub**
- Credentials exposed for **18 months** (2022-2024)
- Attacker discovered via automated GitHub scanning tools
- **$120K** unauthorized cryptocurrency mining costs
- Series B investors discovered breach during due diligence
- **$50M funding round cancelled**
- CTO and CFO terminated

**Root Causes:**
1. No git-secrets pre-commit hook
2. No GitHub secret scanning enabled
3. No key rotation (same key for 18 months)
4. No cost alerts on cloud provider
5. No audit logging of API usage

**Prevention with This Module:**
- ✅ Vault integration (no secrets in code)
- ✅ Quarterly key rotation (max 90-day exposure)
- ✅ Audit logs to S3 Glacier (detect unauthorized usage)
- ✅ Cost monitoring alerts
- ✅ git-secrets pre-commit hook (reject commits with secrets)

#### Failure 2: Unencrypted Database Backups

**What Happened:**
- PostgreSQL backups stored in S3 without encryption
- Backups contained PII for EU citizens (GDPR violation)
- Data residency breach (EU data in US S3 bucket)
- Discovered during GDPR audit
- €20M fine (4% of annual revenue)

**Root Causes:**
1. No encryption at rest for backups
2. No data residency controls
3. Manual backup process (no validation)
4. No encryption verification in deployment checklist

**Prevention:**
- ✅ PostgreSQL TDE (Transparent Data Encryption)
- ✅ S3 SSE-KMS encryption for backups
- ✅ Multi-region Vault (EU backups in EU region)
- ✅ Automated deployment checklist (verify encryption)

#### Failure 3: Expired TLS Certificates

**What Happened:**
- Manual certificate renewal process
- Certificate expired at 2 AM on weekend
- **4-hour production outage** (all HTTPS requests failed)
- 50+ business units affected
- Customer trust loss, SLA penalties

**Root Causes:**
1. Manual renewal (forgot to renew on time)
2. No monitoring/alerts for cert expiry
3. No automated renewal process
4. Single point of failure (one cert for all services)

**Prevention:**
- ✅ cert-manager automation (renews 30 days before expiry)
- ✅ Prometheus alerts (expiry threshold monitoring)
- ✅ Zero-downtime renewal (hot reload)
- ✅ Separate certs per service (isolation)

#### Failure 4: Insufficient Audit Logging

**What Happened:**
- SOX 404 audit discovered missing logs for secret access
- Could not prove who accessed sensitive data
- Audit failure, compliance penalty
- 6-month remediation period

**Root Causes:**
1. Logging not implemented for Vault access
2. No immutable storage (logs in mutable S3)
3. Insufficient log retention (only 90 days, need 7 years)

**Prevention:**
- ✅ audit_log_entry() for ALL operations
- ✅ S3 Glacier with legal hold (immutable)
- ✅ 7-year retention (SOX 404 compliance)
- ✅ Digital signatures for integrity

### Detection Patterns

**Hardcoded Secrets:**
```bash
# git-secrets pattern
git secrets --add 'sk-[a-zA-Z0-9]{32,}'  # OpenAI keys
git secrets --add 'pinecone-[a-zA-Z0-9-]{32,}'  # Pinecone keys
```

**Unencrypted Data:**
```python
# Pre-deployment check
assert postgres_config['ssl_mode'] == 'require'
assert s3_bucket['encryption'] == 'AES256'
```

**Expired Certificates:**
```python
# Prometheus alert
cert_expiry_days < 30 → CRITICAL alert
cert_expiry_days < 7  → PAGE ops team
```

In [ ]:
# Failure Detection Simulation

print("=" * 60)
print("Security Failure Detection Simulation")
print("=" * 60)
print("")

# Test 1: Hardcoded Secret Detection
print("Test 1: Hardcoded Secret Detection")
code_sample = """
# BAD CODE (would be rejected by git-secrets)
OPENAI_API_KEY = "sk-1234567890abcdef"  # ❌ HARDCODED
"""

import re
hardcoded_pattern = r'sk-[a-zA-Z0-9]{20,}'
if re.search(hardcoded_pattern, code_sample):
    print("  ❌ FAILURE: Hardcoded OpenAI API key detected!")
    print("     Would be blocked by git-secrets pre-commit hook")
else:
    print("  ✓ PASS: No hardcoded secrets found")
print("")

# Test 2: Unencrypted Backup Detection
print("Test 2: Unencrypted Database Backup Detection")
backup_config = {
    "encrypted": False,  # ❌ VIOLATION
    "location": "s3://backups/postgres-dump.sql",
    "region": "us-east-1"  # ❌ EU data in US region
}

if not backup_config["encrypted"]:
    print("  ❌ FAILURE: Database backup not encrypted!")
    print("     GDPR/DPDPA violation - would fail compliance audit")
else:
    print("  ✓ PASS: Backup encrypted")
print("")

# Test 3: Missing Audit Events
print("Test 3: Audit Logging Completeness")
logged_events = ["user_login", "secret_access"]  # ❌ Missing key_rotation
required_events = ["user_login", "secret_access", "key_rotation", "encryption_request"]

missing = set(required_events) - set(logged_events)
if missing:
    print(f"  ❌ FAILURE: Missing audit events: {missing}")
    print("     SOX 404 compliance failure")
else:
    print("  ✓ PASS: All required events logged")
print("")

# Test 4: Certificate Expiry
print("Test 4: Certificate Expiry Check")
days_until_expiry = 25  # ❌ Less than 30-day threshold
renewal_threshold = 30

if days_until_expiry < renewal_threshold:
    print(f"  ⚠️  WARNING: Certificate expires in {days_until_expiry} days")
    print("     Renewal recommended (threshold: 30 days)")
    print("     cert-manager would trigger automatic renewal")
else:
    print(f"  ✓ PASS: Certificate valid for {days_until_expiry} days")
print("")

print("=" * 60)
print("Summary: 3 failures detected, 1 warning")
print("All would be caught by automated deployment checks")
print("=" * 60)

# Expected: Multiple security failures detected

## Section 9: Production Deployment Workflow

### Pre-Deployment Checklist

**Infrastructure:**
- [ ] Vault cluster deployed (3-node HA with auto-unseal)
- [ ] Kubernetes cluster ready (1.28+)
- [ ] cert-manager installed (1.13+)
- [ ] PostgreSQL with TDE enabled
- [ ] S3 Glacier bucket for audit logs (legal hold enabled)
- [ ] Prometheus + Grafana for monitoring

**Vault Configuration:**
- [ ] Kubernetes auth method configured
- [ ] RBAC policies created (rag-api, admin roles)
- [ ] Secrets populated (OpenAI, Pinecone, PostgreSQL, KEK)
- [ ] Database secrets engine configured
- [ ] Audit logging enabled

**Application:**
- [ ] Environment variables configured (.env)
- [ ] VAULT_ENABLED=true
- [ ] ServiceAccount created in K8s
- [ ] RBAC role binding verified
- [ ] Tests passing (pytest -v)

### Deployment Steps

```bash
# 1. Deploy Vault cluster
helm install vault hashicorp/vault \
  --set server.ha.enabled=true \
  --set server.ha.replicas=3

# 2. Initialize Vault
vault operator init
vault operator unseal  # Repeat 3 times with different keys

# 3. Configure Kubernetes auth
vault auth enable kubernetes
vault write auth/kubernetes/config \
  kubernetes_host="https://kubernetes.default.svc"

# 4. Create RBAC policy
vault policy write rag-api-policy rag-api-policy.hcl

# 5. Bind policy to ServiceAccount
vault write auth/kubernetes/role/rag-api \
  bound_service_account_names=rag-api \
  bound_service_account_namespaces=production \
  policies=rag-api-policy \
  ttl=24h

# 6. Deploy application
kubectl apply -f k8s/deployment.yaml

# 7. Verify Vault authentication
kubectl logs -f deployment/rag-api | grep "Vault authenticated"

# 8. Test secret retrieval
kubectl exec -it deployment/rag-api -- python -c \
  "from src.l3_m2_security_access_control import VaultClient; \
   v = VaultClient(...); \
   print(v.get_openai_key()[:10])"
```

### Post-Deployment Validation

```bash
# 1. Check Vault status
vault status
# Expected: sealed=false, ha_mode=active

# 2. Test API endpoints
curl https://rag-api.gcc.techvoyagehub.com/
# Expected: {"status": "healthy", "vault_authenticated": true}

# 3. Verify certificate
openssl s_client -connect rag-api.gcc.techvoyagehub.com:443 \
  -showcerts | grep "Issuer: Let's Encrypt"

# 4. Check audit logs
aws s3 ls s3://gcc-audit-logs/vault/ --recursive
# Expected: Recent log files present

# 5. Test key rotation
curl -X POST https://rag-api.gcc.techvoyagehub.com/keys/rotate \
  -H "Content-Type: application/json" \
  -d '{"service": "openai", "new_key": "sk-test-rotation-xyz"}'
# Expected: {"status": "success"}
```

### Monitoring & Alerts

**Prometheus Metrics:**
- `vault_core_unsealed` (should be 1)
- `vault_core_active` (should be 1)
- `cert_expiry_days` (alert if < 30)
- `secret_access_count` (detect anomalies)
- `key_rotation_last_timestamp` (alert if > 95 days)

**Grafana Dashboards:**
- Vault cluster health
- Secret access patterns
- Certificate expiry timeline
- Key rotation schedule
- Audit log volume

In [ ]:
# Production Readiness Check

print("=" * 60)
print("Production Deployment Readiness Assessment")
print("=" * 60)
print("")

# Simulate deployment checks
checks = {
    "Vault cluster (3-node HA)": False,  # Set to True when deployed
    "Kubernetes auth configured": False,
    "RBAC policies created": False,
    "Secrets populated in Vault": False,
    "cert-manager installed": False,
    "PostgreSQL TDE enabled": False,
    "S3 Glacier audit logs": False,
    "Tests passing": True,  # We know tests pass from pytest
    "Environment variables configured": True,
}

print("Infrastructure Checks:")
for check, status in checks.items():
    symbol = "✓" if status else "❌"
    print(f"  {symbol} {check}")
print("")

passed = sum(checks.values())
total = len(checks)
percentage = (passed / total) * 100

print(f"Readiness Score: {passed}/{total} ({percentage:.0f}%)")
print("")

if percentage == 100:
    print("✓ READY FOR PRODUCTION DEPLOYMENT")
elif percentage >= 80:
    print("⚠️  ALMOST READY - Complete remaining checks")
else:
    print("❌ NOT READY - Significant work required")
print("")

print("Next Steps:")
if not checks["Vault cluster (3-node HA)"]:
    print("  1. Deploy Vault cluster: helm install vault hashicorp/vault")
if not checks["Kubernetes auth configured"]:
    print("  2. Configure K8s auth: vault auth enable kubernetes")
if not checks["Secrets populated in Vault"]:
    print("  3. Populate secrets: vault kv put gcc-secrets/...")
if not checks["S3 Glacier audit logs"]:
    print("  4. Create S3 Glacier bucket with legal hold")

# Expected: Deployment readiness assessment with actionable next steps

## Section 10: Summary & Next Steps

### What You've Learned

**Core Concepts:**
1. ✅ **HashiCorp Vault Integration:** Dynamic secrets retrieval with K8s ServiceAccount auth (no hardcoded credentials)
2. ✅ **AES-256 Encryption at Rest:** Envelope encryption with KEK in Vault, random DEK per operation
3. ✅ **TLS 1.3 Encryption in Transit:** Automated cert-manager + Let's Encrypt (zero-downtime renewal)
4. ✅ **Automated Key Rotation:** Quarterly for APIs, daily for DB credentials, annual for KEK
5. ✅ **GCC Compliance:** SOX 404, DPDPA, GDPR, SOC 2, ISO 27001 requirements
6. ✅ **Failure Prevention:** Detection and mitigation of hardcoded secrets, unencrypted backups, expired certs

### Real-World Impact

**What This Module Prevents:**
- ❌ $50M funding loss from hardcoded secrets (Bangalore GCC case)
- ❌ €20M GDPR fines from unencrypted backups
- ❌ 4-hour production outages from expired certificates
- ❌ SOC 2 audit failures from insufficient logging
- ❌ Data residency violations (DPDPA, GDPR)

**What This Module Enables:**
- ✅ Multi-tenant RAG serving 50+ business units across 3 regions
- ✅ Zero hardcoded secrets (all from Vault with 24-hour TTL)
- ✅ Full encryption compliance (SOC 2, ISO 27001, PCI-DSS)
- ✅ Automated key rotation (quarterly/daily with zero downtime)
- ✅ Immutable audit trails (7-year retention for SOX 404)

### Architecture Recap

```
User Request
  ↓ TLS 1.3 (cert-manager + Let's Encrypt)
RAG API Pod
  ↓ K8s ServiceAccount auth (24-hour token)
Vault Cluster (3-node HA)
  ↓ Dynamic secrets (OpenAI, Pinecone, PostgreSQL, KEK)
Application
  ↓ AES-256 encryption (envelope: DEK + KEK)
Pinecone / PostgreSQL
  ↓ Audit logging
S3 Glacier (7-year retention, immutable)
```

### Next Module: L3 M2.4

**Rate Limiting & DDoS Protection**
- Token bucket rate limiting per tenant
- IP-based throttling with Redis
- Distributed rate limiting (50+ business units)
- Cost control (prevent $120K mining attacks)
- Fair resource allocation across tenants

### Additional Resources

**HashiCorp Vault:**
- Official docs: https://developer.hashicorp.com/vault
- Kubernetes auth guide: https://developer.hashicorp.com/vault/docs/auth/kubernetes
- Dynamic secrets: https://developer.hashicorp.com/vault/docs/secrets/databases

**Compliance:**
- SOX 404: https://www.soxlaw.com/
- DPDPA (India): https://www.meity.gov.in/data-protection-framework
- GDPR: https://gdpr.eu/
- SOC 2: https://www.aicpa.org/soc

**Encryption:**
- NIST AES-256: https://csrc.nist.gov/publications/detail/fips/197/final
- Envelope encryption: https://cloud.google.com/kms/docs/envelope-encryption
- TLS 1.3: https://datatracker.ietf.org/doc/html/rfc8446

### Practice Exercises

1. **Deploy Vault Dev Server:** `vault server -dev` and configure K8s auth
2. **Rotate All Keys:** Implement full rotation workflow (API keys + DB creds + KEK)
3. **Multi-Region Setup:** Deploy Vault clusters in 3 regions with replication
4. **Audit Log Analysis:** Query S3 Glacier logs for compliance reporting
5. **Failure Simulation:** Test recovery from expired certs, lost Vault tokens

### Success Criteria

You've mastered this module when you can:
- [ ] Deploy 3-node HA Vault cluster with K8s auth
- [ ] Retrieve secrets dynamically (no hardcoded credentials)
- [ ] Implement AES-256 envelope encryption with Vault KEK
- [ ] Configure cert-manager for TLS 1.3 automation
- [ ] Build automated key rotation (quarterly/daily/annual)
- [ ] Generate compliance audit reports (SOX 404, GDPR)
- [ ] Explain Bangalore GCC $50M failure and prevention
- [ ] Deploy multi-region Vault for data residency

In [ ]:
# Final Summary

print("=" * 60)
print("L3 M2.3: Encryption & Secrets Management - Complete!")
print("=" * 60)
print("")

print("Key Takeaways:")
print("  1. Zero hardcoded secrets (all from Vault with 24h TTL)")
print("  2. AES-256 envelope encryption (KEK in Vault, random DEK)")
print("  3. TLS 1.3 automation (cert-manager + Let's Encrypt)")
print("  4. Automated key rotation (quarterly/daily/annual)")
print("  5. Multi-framework compliance (SOX, DPDPA, GDPR, SOC 2)")
print("  6. Immutable audit trails (7-year S3 Glacier retention)")
print("")

print("Production Capabilities Unlocked:")
print("  ✓ 50+ tenant multi-region RAG system")
print("  ✓ SOC 2 / ISO 27001 certification ready")
print("  ✓ Zero-downtime key rotation")
print("  ✓ Data residency compliance (India, US, EU)")
print("  ✓ Prevention of $50M+ security failures")
print("")

print("Next Steps:")
print("  1. Review README.md for deployment checklist")
print("  2. Run production tests: pytest -v tests/")
print("  3. Deploy Vault dev server for practice")
print("  4. Proceed to L3 M2.4: Rate Limiting & DDoS Protection")
print("")

print("Questions? Issues?")
print("  GitHub: https://github.com/yesvisare/gcc_comp_ai_ccc_l2/issues")
print("  Track: https://techvoyagehub.com/l3-production-rag")
print("")

print("=" * 60)
print("Happy Securing! 🔒")
print("=" * 60)

# Expected: Completion summary with next steps